In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import urllib.request, io
from PIL import ImageTk, Image
from pytube import YouTube
import os
import threading
import subprocess

In [ ]:
# A function to return the full path of the files.
def resource_path(relative_path):
    try:
        base_path = sys._MEIPASS
    except Exception:
        base_path = os.path.abspath(".")

    return os.path.join(base_path, relative_path)

def get_songs(url):
    yt = YouTube(url)
    video = yt.streams.filter(only_audio=True).first()
    downloaded_file = video.download()
    base, ext = os.path.splitext(downloaded_file)
    new_file = base + '.mp3'
    os.rename(downloaded_file, new_file)
    print("Done")

In [ ]:
class CEntry(tk.Entry):
    def __init__(self, parent, *args, **kwargs):
        tk.Entry.__init__(self, parent, *args, **kwargs)

        self.changes = [""]
        self.steps = int()

        self.bind("<Control-z>", self.undo)
        self.bind("<Control-y>", self.redo)

        self.bind("<Key>", self.add_changes)

    def undo(self, event=None):
        if self.steps >= 1:
            self.steps -= 1
            self.delete(-1, 'end')
            self.insert('end', self.changes[self.steps])

    def redo(self, event=None):
        if self.steps < len(self.changes):
            self.delete(-1, 'end')
            self.insert('end', self.changes[self.steps])
            self.steps += 1

    def add_changes(self, event=None):
        if self.get() != self.changes[-1]:
            self.changes.append(self.get())
            self.steps += 1

In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import urllib.request, io
from PIL import ImageTk, Image
from pytube import YouTube
import os
import threading
import subprocess
from itertools import count, cycle

# A function to return the full path of the files.
def resource_path(relative_path):
    try:
        base_path = sys._MEIPASS
    except Exception:
        base_path = os.path.abspath(".")

    return os.path.join(base_path, relative_path)

class CEntry(tk.Entry):
    def __init__(self, parent, *args, **kwargs):
        tk.Entry.__init__(self, parent, *args, **kwargs)

        self.changes = [""]
        self.steps = int()

        self.bind("<Control-z>", self.undo)
        self.bind("<Control-y>", self.redo)

        self.bind("<Key>", self.add_changes)

    def undo(self, event=None):
        if self.steps >= 1:
            self.steps -= 1
            self.delete(-1, 'end')
            self.insert('end', self.changes[self.steps])

    def redo(self, event=None):
        if self.steps < len(self.changes):
            self.delete(-1, 'end')
            self.insert('end', self.changes[self.steps])
            self.steps += 1

    def add_changes(self, event=None):
        if self.get() != self.changes[-1]:
            self.changes.append(self.get())
            self.steps += 1
            

win = tk.Tk()
win.title('YouTube Downloader')

# root = tk.Tk()
# lbl = ImageLabel(win)
# lbl.place(relx=0.25, rely=0.55, relwidth=0.5)
# lbl.load('loading_circle.gif')

win_width = 400
win_height = 300

screen_width = win.winfo_screenwidth()
screen_height = win.winfo_screenheight()

x_cord = (screen_width/2) - (win_width/2)
y_cord = (screen_height/2) - (win_height/2)
win.geometry(f'{win_width}x{win_height}+{int(x_cord)}+{int(y_cord)}')

style = ttk.Style()
style.theme_use('clam')
style.configure('TButton', anchor='center', font=('calibri', 12), focuscolor='#CD322D')
style.map('TButton', foreground=[('!disabled', 'red'), ('active', 'red')],
          background=[('!disabled', 'white'), ('active', 'white')],
          bordercolor=[('!disabled', '#CD322D'), ('active', '#CD322D')],
          borderwidth=[('!disabled', 0), ('active', 0)])


canvas = tk.Canvas(win, width=400, height=300)
canvas.pack(fill='both', expand=True)

DELAY = 80

def animate():
    global img
#     if submit_thread.is_alive():
    img = canvas.create_image(175, 20, anchor='nw',image=next(frames_cycle))
    canvas.itemconfig(img,image=next(frames_cycle))
#         win.after(DELAY,animate)

def delete():
    canvas.delete(img)

url_entry = CEntry(win)
url_entry.config(highlightthickness=2, highlightbackground='#CD322D', highlightcolor='#CD322D')
url_entry.place(relx=0.25, rely=0.75, relwidth=0.5)

frames = []
im = Image.open('loading_red.gif')
for frame in range(im.n_frames):
    im.seek(frame)
    frames.append(ImageTk.PhotoImage(im.copy()))
frames_cycle=cycle(frames)

canvas.pack()
picture = tk.PhotoImage(file=resource_path('youtube logo.png'), master=win)
canvas.create_image(0, 0, image=picture, anchor='nw')

canvas.create_text(80,235,fill='white',font='calibri 13 bold',
                        text='URL:')

url_entry = CEntry(win)
url_entry.config(highlightthickness=2, highlightbackground='#CD322D', highlightcolor='#CD322D')
url_entry.place(relx=0.25, rely=0.75, relwidth=0.5)

# # Creating a right-click menu.
link_entry_place_menu = tk.Menu(win, tearoff = False) 
link_entry_place_menu.add_command(label="גזור", accelerator="Ctrl+X",
              command=lambda: link_entry_place_menu.focus_get().event_generate('<<Cut>>')) 
link_entry_place_menu.add_command(label ="העתק", accelerator="Ctrl+C",
              command=lambda: link_entry_place_menu.focus_get().event_generate('<<Copy>>'))
link_entry_place_menu.add_command(label ="הדבק", accelerator="Ctrl+V",
              command=lambda: link_entry_place_menu.focus_get().event_generate('<<Paste>>')) 
link_entry_place_menu.add_command(label ="בחירת הכל", accelerator="Ctrl+A",
              command=lambda: link_entry_place_menu.focus_get().event_generate('<<SelectAll>>')) 

link_entry_place_menu['bg'] = 'white'

# A function to operate the right-click menu of the login window.
def do_popup_for_login(event): 
    try: 
        link_entry_place_menu.tk_popup(event.x_root, event.y_root) 
    finally: 
        link_entry_place_menu.grab_release() 

# Binding the function to the right-click menu.
url_entry.bind("<Button-3>", do_popup_for_login)
url_entry.bind("<Ctrl-Z>", url_entry.event_generate('<<Undo>>'))
# url_entry

def callback(event):
    if (event.state & 4 > 0):
        keys = "Ctrl-%s" % chr(event.keycode)
        if keys == 'Ctrl-C':
            url_entry.event_generate('<<Copy>>')
        elif keys == 'Ctrl-V':
            url_entry.event_generate('<<Paste>>')
        elif keys == 'Ctrl-A':
            url_entry.event_generate('<<SelectAll>>')
        elif keys == 'Ctrl-X':
            url_entry.event_generate('<<Cut>>')
        elif keys == 'Control-Z':
            url_entry.event_generate('<<Undo>>')
        elif keys == 'Control-Y':
            url_entry.event_generate('<<Redo>>')
    
win.bind('<Key>', callback)

# Loading bar function to indicate that the weekly schedule is being organaized.
def loading_bar():
    progress_bar['value'] += 0
    progress_bar['value'] += 0.05
    win.update_idletasks()
    
    
# This function will let the code run without freezing the GUI and creating the loading bar
# once you pressed the "Create Worksheet" button.
def new_thread_for_song_download():
    global submit_thread
    global progress_bar
    submit_thread = threading.Thread(target=get_songs)
    submit_thread.daemon = True
    submit_thread.start()
    win.after(20, check_when_finish_downloading)
    delete()
    
#     style = ttk.Style()
#     style.configure("TProgressbar", background='#CD322D', troughcolor='white',
#                         bordercolor='white', lightcolor='#CD322D', darkcolor='#CD322D')

#     # Creating a progress bar to indicate that the hours are loading into the google sheets.
#     progress_bar = ttk.Progressbar(win, orient = 'horizontal',
#                 length =300, mode = 'determinate')
#     progress_bar.place(relx=0.18, rely=0.2,  width=250, height=15)
    
# This function will check every 20 ms if the worksheet is still being prepared
# and when it will finish - a pop-up message will say it's done!
def check_when_finish_downloading():
    if submit_thread.is_alive():
        win.after(20, check_when_finish_downloading)
        animate()
#         loading_bar() 
    else:
        delete()
#         frames
#         picture = tk.PhotoImage(file=resource_path('youtube logo.png'), master=win)
#         canvas.create_image(0, 0, image=picture, anchor='nw')

#         canvas.create_text(80,235,fill='white',font='calibri 13 bold',
#                                 text='URL:')

url_entry = CEntry(win)
url_entry.config(highlightthickness=2, highlightbackground='#CD322D', highlightcolor='#CD322D')
url_entry.place(relx=0.25, rely=0.75, relwidth=0.5)
        
# #         progress_bar.destroy()
        
def get_songs():
    destination = r'C:\Users\User\Downloads'
    yt = YouTube(url_entry.get())
    video = yt.streams.filter(only_audio=True).first()
    if f'{video.title}.mp3' in os.listdir(destination):
        delete()
        tk.messagebox.showinfo('Song Exists', f'{video.title} Already Exists In Your Computer!')
    else:
        downloaded_file = video.download(output_path=destination)
        base, ext = os.path.splitext(downloaded_file)
        new_file = base + '.mp3'
        os.rename(downloaded_file, new_file)
        delete()
        tk.messagebox.showinfo('Download Complete', f'{video.title} Was download Succsefully!')


btn = ttk.Button(master=win, text='Download', cursor='hand2', command=new_thread_for_song_download)
btn.place(relx=0.392, rely=0.85, relwidth=0.2, relheight=0.1)



win.resizable(False, False)
win.iconbitmap(resource_path('youtube_logo_for_icon.ico'))
win.mainloop()

In [ ]:
from PIL import Image
import tkinter as tk   # for Python3 use tkinter
def show_R(self):
    win = Toplevel()
    canvas = Canvas(win, width=1380, height=800, bg='White')
    canvas.pack(expand=True)
    # Maximum frames 
    numIdx = 28
    frames = [PhotoImage(file='D:/Python_code/tk/final/gif/R_fianl.gif',
    format='gif -index %i' % (i)) for i in range(numIdx)]
    # Fill the frame content to frames
def update(idx): # Timer Functions 
    frame = frames[idx]
    idx += 1 # The sequence number of the next frame ：
    canvas.create_image(15, 15, anchor=NW, image=frame)
    # label.configure(image = frame) # Show label Method the picture of the current frame 
    # 0.1 second (500 millisecond ) Then continue to execute the timer function (update)
    if self.isplaying:
    self.root.after(self.time, update, idx % numIdx)
    else:
    self.stop_idx = idx
    win.after(0, update(0))
win.mainloop()

In [ ]:
import tkinter as tk
from PIL import Image, ImageTk
from itertools import count
import os, sys

# A function to return the full path of the files.
def resource_path(relative_path):
    try:
        base_path = sys._MEIPASS
    except Exception:
        base_path = os.path.abspath(".")

    return os.path.join(base_path, relative_path)

class ImageLabel(tk.Label):
    """a label that displays images, and plays them if they are gifs"""
    def load(self, im):
#         if isinstance(im):
        im = Image.open(im)#tk.PhotoImage(file=resource_path('youtube logo.png'), master=c)
#         c.create_text(0, 0, text=im, anchor='nw')
        self.loc = 0
        self.frames = []

        try:
            for i in count(1):
                self.frames.append(ImageTk.PhotoImage(im.copy()))
                im.seek(i)
        except EOFError:
            pass

#         c.create_image(0, 0, image=tk.PhotoImage(im), anchor='nw')
        try:
            self.delay = im.info['duration']
        except:
            self.delay = 100

        if len(self.frames) == 1:
            c.create_image(200, 0, image=self.frames[0], anchor='nw')
#             self.config(image=self.frames[0])
        else:
            self.next_frame()
#         c.create_image(0, 0, image=ImageTk.PhotoImage(im), anchor='s')

    def unload(self):
        self.config(image="")
        self.frames = None

    def next_frame(self):
        if self.frames:
            self.loc += 1
            self.loc %= len(self.frames)
            self.config(image=self.frames[self.loc])
            self.after(self.delay, self.next_frame)

root = tk.Tk()
c = tk.Canvas(root)
lbl = ImageLabel()
lbl.pack()
lbl.load('loading_circle.gif')
root.mainloop()

In [ ]:
import tkinter as tk
from PIL import ImageTk, Image
from itertools import count

root = tk.Tk()
root.geometry('300x300')
canvas = tk.Canvas(root, width=300, height=300, bg='red', highlightthickness=0)

images_list = []
gif_duration = ''
images_counting = -1

# canvas = tk.Canvas(root, width=300, height=300)
canvas.pack(fill='both', expand=True)

def extract_images_from_gif(path):
    global gif_duration
    image = Image.open(path)

    for i in count(1):
        try:
            images_list.append(ImageTk.PhotoImage(image.copy()))
#             PhotoImage.put('blue', (150,150))
            image.seek(i)
            
        except Exception as error:
            print(error)
            break

    gif_duration = int(image.info['duration'])
    
def play_slider():
    global images_counting
    try:
        images_counting += 1
        
        
        canvas.create_image(150,150, image=images_list[images_counting], anchor='center')
#         slider_lb.config(image=images_list[images_counting])
        
        root.after(gif_duration, play_slider)
        
    except Exception as error:
#         print(error)
        images_counting = -1
        root.after(gif_duration, play_slider)

extract_images_from_gif('loading.gif')
root.after(1000, play_slider)
root.mainloop()

In [ ]:
import numpy as np
plate=np.zeros([12, 17], dtype=np.uint32)

def plate_animation(): 
    color = compute_colors("red3", "blue", len(plate[0]))
    for o in range(len(plate)):
        for p in range(len(plate[0])):
            canvas.itemconfig(plate[o][p], fill=color[p])

In [11]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import urllib.request, io
from PIL import ImageTk, Image
from pytube import YouTube
import os
import threading
import subprocess
from itertools import count, cycle

DELAY = 20

# A function to return the full path of the files.
def resource_path(relative_path):
    try:
        base_path = sys._MEIPASS
    except Exception:
        base_path = os.path.abspath(".")

    return os.path.join(base_path, relative_path)

class CEntry(tk.Entry):
    def __init__(self, parent, *args, **kwargs):
        tk.Entry.__init__(self, parent, *args, **kwargs)

        self.changes = [""]
        self.steps = int()

        self.bind("<Control-z>", self.undo)
        self.bind("<Control-y>", self.redo)

        self.bind("<Key>", self.add_changes)

    def undo(self, event=None):
        if self.steps >= 1:
            self.steps -= 1
            self.delete(-1, 'end')
            self.insert('end', self.changes[self.steps])

    def redo(self, event=None):
        if self.steps < len(self.changes):
            self.delete(-1, 'end')
            self.insert('end', self.changes[self.steps])
            self.steps += 1

    def add_changes(self, event=None):
        if self.get() != self.changes[-1]:
            self.changes.append(self.get())
            self.steps += 120

def animate():
    global img
    img = canvas.create_image(120, 50, anchor='nw',image=next(frames_cycle))
    canvas.itemconfig(img,image=next(frames_cycle))
    root.after(DELAY,animate)

def delete():
    canvas.delete(img)

root = tk.Tk()

root_width = 400
root_height = 300

screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

x_cord = (screen_width/2) - (root_width/2)
y_cord = (screen_height/2) - (root_height/2)
root.geometry(f'{root_width}x{root_height}+{int(x_cord)}+{int(y_cord)}')

canvas = tk.Canvas(root, width=400, height=300)
picture = tk.PhotoImage(file=resource_path('youtube logo.png'), master=root)
canvas.create_image(0, 0, image=picture, anchor='nw')

style = ttk.Style()
style.theme_use('clam')
style.configure('TButton', anchor='center', font=('calibri', 12), focuscolor='#CD322D')
style.map('TButton', foreground=[('!disabled', 'red'), ('active', 'red')],
          background=[('!disabled', 'white'), ('active', 'white')],
          bordercolor=[('!disabled', '#CD322D'), ('active', '#CD322D')],
          borderwidth=[('!disabled', 0), ('active', 0)])

# root.geometry('400x300')
# root.config(background='#CD322D')
frames = []
im = Image.open('loading.gif')
for frame in range(im.n_frames):
    im.seek(frame)
    frames.append(ImageTk.PhotoImage(im.convert('RGBA')))
frames_cycle=cycle(frames)

# canvas = tk.Canvas(root,bg='#CD322D', width=400, height=300)
# picture = tk.PhotoImage(file=resource_path('youtube logo.png'), master=root)
# canvas.create_image(0, 0, image=picture, anchor='nw')



canvas.create_text(80,235,fill='white',font='calibri 13 bold',
                        text='URL:')

tk.Button(root, text='Delete', command=delete).place(relx=0.4, rely=0.5, relwidth=0.2)
tk.Button(root, text='create', command=animate).place(relx=0.4, rely=0.7, relwidth=0.2)
# img = canvas.create_image(120, 50, anchor='nw',image=next(frames_cycle))
canvas.pack(fill='both', expand=True)
# animate()
root.resizable(False, False)
root.mainloop()

In [2]:
import tkinter as tk
from PIL import Image, ImageTk
from itertools import cycle

DELAY = 20

def animate():
    canvas.itemconfig(img,image=next(frames_cycle))
    root.after(DELAY,animate)
    
def dele

root = tk.Tk()
root.geometry('300x300')
root.config(background='red')

frames = []
im = Image.open('loading.gif')
for frame in range(im.n_frames):
    im.seek(frame)
    frames.append(ImageTk.PhotoImage(im.convert('RGBA')))
frames_cycle=cycle(frames)

canvas = tk.Canvas(root,bg='red', width=300, height=300)
img = canvas.create_image(120,0,anchor='nw',image=next(frames_cycle))
canvas.pack()
animate()
root.resizable(False, False)
root.mainloop()